Il ruolo dell'attaccante è fondamentale nel gioco del calcio. E' compito suo infatti fare gol ma non solo, è anche un'ottima chiave di gioco per la creazione degli spazi utili per i compagni di squadra o anche per difendere il pallone in una zona di campo lontana dalla propria area e permettere così al team di rifiatare e alzare il baricentro. In questa sezione ci occupiamo della parte 'demografica' degli attaccanti di Serie A. Andremo ad analizzare come sono distribuite le età, le nazionalità ed i minuti giocati, per gli attaccanti. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import scipy.stats as stats
import tensorflow as tf
import funzioni
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from matplotlib.widgets import Slider
import pandas as pd
from math import sqrt 

pd.set_option('display.max_columns', None)

# Elenco dei file da leggere
files = ['attaccanti_23.xlsx', 'attaccanti_22.xlsx', 'attaccanti_21.xlsx', 'attaccanti_20.xlsx', 'attaccanti_19.xlsx']

# Dizionario per mappare i file agli anni
year_mapping= {'attaccanti_23.xlsx': 2023,'attaccanti_22.xlsx': 2022,'attaccanti_21.xlsx': 2021,'attaccanti_20.xlsx': 2020,
                'attaccanti_19.xlsx': 2019}

In [ ]:
# Cella utile inserita in caso ci fossero aggiornamenti delle funzioni da importare

import importlib
importlib.reload(funzioni)

In [ ]:
# Definisco fasce di età e etichette
bins = [15, 20, 25, 30, 35, 40]
labels = ['15-20', '21-25', '26-30', '31-35', '36-40']

# Dizionario per conservare i conteggi annuali per ciascuna fascia d'età
fasce_eta_conti = {label: [] for label in labels}
medie_totali_annuali = []
deviazioni_annuali = []

# Ciclo sui file per calcolare la numerosità per ciascuna fascia, la media totale e la deviazione standard
for file in files:
    data = pd.read_excel(file)
    eta = data['Age']

    # Categorizzazione delle età nelle fasce specificate
    eta_bins = pd.cut(eta, bins=bins, labels=labels, right=True, include_lowest=True)

    # Conteggio il numero di giocatori per ciascuna fascia di età
    for label in labels:
        conteggio_eta_fascia = (eta_bins == label).sum()
        fasce_eta_conti[label].append(conteggio_eta_fascia)

    # Calcolo media e deviazione standard
    medie_totali_annuali.append(eta.mean())
    deviazioni_annuali.append(eta.std(ddof=1))

anni = ['2019', '2020', '2021', '2022', '2023']  

# Creazione del grafico a barre
fig, ax = plt.subplots(figsize=(10, 8))
mini_bar_width = 0.8 / len(anni) 
x_pos = np.arange(len(labels))

for i, anno in enumerate(anni):
    conti_annuali = [fasce_eta_conti[label][i] for label in labels]
    offset = (i - (len(anni) - 1) / 2) * mini_bar_width
    ax.bar(x_pos + offset, conti_annuali, mini_bar_width, label=anno)

ax.set_xlabel('Fascia di Età')
ax.set_ylabel('Numero di Giocatori')
ax.set_title('Numero di Attaccanti per Fascia di Età e Anno')
ax.set_xticks(x_pos)
ax.set_xticklabels(labels)
ax.legend(title='Anno')

ax.set_yticks(np.arange(0, max(max(fasce_eta_conti.values())) + 5, 5))

plt.show()

# 2° Grafico
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(anni, medie_totali_annuali, marker='o', color='k', label='Media Totale')

# Barre di errore
ax.errorbar(anni, medie_totali_annuali, yerr=deviazioni_annuali, fmt='o', color='k', label='Deviazione Standard')

ax.set_xlabel('Anno')
ax.set_ylabel('Media Età')
ax.set_title("Andamento della Media Totale dell'Età dal 2019 al 2023")

ax.set_ylim(15, 40)
ax.set_yticks(np.arange(15, 40, 1))

ax.grid(True)
ax.legend()

plt.show()

Dopo aver importato i file necessari andiamo ad analizzare come sono variate le età nei 5 anni analizzati, visualizziamo il tutto con una serie di istogrammi riferiti alle varie fasce di età. Si nota come il valore compreso tra i 21 e i 25 anni sia il più diffuso nel ruolo, seguito da 26-30 e 15-20. Da tenere in considerazione inoltre l'aumento progressivo dell'utilizzo sempre maggiore di giovanissimi, i quali vengono fatti esordire prematuramente. Dal 2020 al 2023 infatti il dato è in costante aumento, e ciò si conferma anche con i discorsi molto ripresi nel corso di questi anni, dove la maggior parte delle squadre di Serie A provano a ringiovanire l'organico e a ricercare sempre più giovani talenti da valorizzare e rivendere ad un prezzo maggiore.
Il secondo grafico, riferito alla media di età nel corso degli anni, si nota una error bar notevole, che dipende proprio dalle grandi differenze di età che si possono trovare nel parco attaccanti.

Passiamo ora all'analisi delle nazionalità. Vediamo quali sono le nazionalità più presenti in Serie A utilizzando il concept precedente, in modo da avere un confronto diretto anno per anno.

In [ ]:
# Conteggio delle nazionalità per ciascun anno
all_counts = {}
for file in files:
    data = pd.read_excel(file)
    year = year_mapping[file]
    nazionalita_counts = data['Nation'].value_counts()
    all_counts[year] = nazionalita_counts

# Unione dei dati in un DataFrame
df_counts = pd.DataFrame(all_counts).fillna(0)

# Creazione di un grafico a barre
fig = go.Figure()

for year in df_counts.columns:
    fig.add_trace(go.Bar( x=df_counts.index, y=df_counts[year], name=str(year)))

# Barra di scorrimento così da favorire la visualizzazione
fig.update_layout(title="Distribuzione delle Nazionalità degli attaccanti per gli anni 2019-2023",
                  xaxis=dict(title="Nazionalità", tickangle=45, automargin=True,rangeslider=dict(visible=True) ),   
                  yaxis=dict( title="Numero di calciatori", tick0=0, dtick=5),barmode='group', width=2000)

fig.show()

Come da aspettativa, la nazionalità italiana risulta nettamente la più diffusa, questo appunto per via del campionato preso in esame. Risultano sorprendenti invece i dati relativi ad Argentina, Brasile e Costa d'Avorio, le quali hanno un indice molto alto e distaccato rispetto alle altre nazionalità.

In [ ]:
statistiche_minuti_per_anno = {}

for file in files:
    data = pd.read_excel(file)
    valori = data['MP']  
    
    # Filtraggio i dati per includere solo i calciatori con più di 20 minuti giocati
    valori_filtrati = valori[valori > 20]

    media_valori_filtrati = valori_filtrati.mean()
    deviazione_standard = valori_filtrati.std()
    
    # Aggiunta dei valori al dizionario
    year = year_mapping[file]
    statistiche_minuti_per_anno[year] = (media_valori_filtrati, deviazione_standard)

# Creazione di un DataFrame per le statistiche dei minuti giocati per anno
df_statistiche = pd.DataFrame( [(year, stat[0], stat[1]) for year, stat in statistiche_minuti_per_anno.items()], 
                              columns=['Anno', 'Media Minuti', 'Deviazione Standard'])

# Ordinamento del DataFrame per anno
df_statistiche = df_statistiche.sort_values('Anno')

plt.figure(figsize=(10, 6))

# Errorbar

plt.errorbar( df_statistiche['Anno'], df_statistiche['Media Minuti'], yerr=df_statistiche['Deviazione Standard'], fmt='o',
              ecolor='k', capsize=5, color='k', label='Media Minuti Giocati')
plt.plot(df_statistiche['Anno'], df_statistiche['Media Minuti'], color='k', linestyle='-', linewidth=2, label='Linea di Tendenza')

# Personalizzazione i tick dell'asse x per mostrare solo gli anni
plt.xticks(df_statistiche['Anno'], fontsize=12)

plt.xlabel('Anno', fontsize=14)
plt.ylabel('Media Minuti Giocati', fontsize=14)
plt.title('Media dei Minuti Giocati dai Calciatori per Anno con Barre di Errore', fontsize=16)

plt.grid(True)

plt.legend()

plt.tight_layout()

plt.show()

In questi grafici vediamo la distribuzione dei minuti giocati diviso 90 dagli attaccanti. In altre parole calcoliamo quante partite complete vengono giocate dagli attaccanti. Abbiamo messo un bound inferiore di 20 minuti giocati per evitare che i dati vengano sfalsati da giocatori con pochi minuti giocati. Vediamo che la distribuzione e la media rimane più o meno la stessa nel corso degli anni. La media addirittura oscilla fra 29.42 e 29.98. Dal grafico si ottiene anche un insolito calo dei minuti giocati nel 2022, probabilmente dovuto dalla presenza di numerose squadre con un assetto ad attaccanti a 2 e non a 3 come negli anni precedenti. Considerando la possibilità di 5 sostituzioni per partita per ogni squadra, si giustificano gli scarti del grafico dal fatto che l'impiego dei giocatori subentrati dalla panchina, i quali hanno un minutaggio per partita considerevolmente minore rispetto ai giocatori titolari.

Aggiungiamo una cella di codice che stampa gli istogrammi di tutte le statistiche di tutti 5 gli anni. In modo da poterli 
analizzare singolarmente in seguito, senza appesantire il codice con tutti i grafici.

In [ ]:
from funzioni import freedman_diaconis_bins
for file in files:
    data = pd.read_excel(file)

    # Selezione solo delle colonne numeriche
    numeric_data = data.select_dtypes(include=[np.number])

    # Creazione di istogrammi per ogni colonna numerica
    for column in numeric_data.columns:
        col_data = numeric_data[column].dropna()  

        if col_data.nunique() <= 1:
            continue

        # Calcolo dei bin
        bins = freedman_diaconis_bins(col_data)

        # Istogramma
        plt.figure(figsize=(10, 6))
        plt.hist(col_data, bins=bins, color='k', edgecolor='r', label='Dati')

        plt.title(f'Istogramma della colonna: {column}')
        plt.xlabel(column)
        plt.ylabel('Numero di giocatori')
        #plt.show()
        plt.close()
        # Calcolo della media e sua stampa a video
        #media = col_data.mean()
        #print(f"La media di {column} per l'anno {year_mapping[file]} è: {media:.2f}")

# Le ultime due righe sono state commentate perche altrimenti il codice avrebbe stampato grafico e media di ogni statistica 
# per tutti i 5 gli anni

Creiamo una funzione che, dati in input la statistica e l'anno desiderato, restituisce l'istogramma di quella statistica
relativa a quell'anno

Con questa funzione possiamo richiamare dei grafici specifici e analizzarli se ci interessa.

In [ ]:
from funzioni import istogramma
statistica = 'Gls'
anno = 2020
istogramma(statistica, anno, year_mapping)

CORRELAZIONI

INTRODUZIONE

Il calcio moderno è diventato molto caro, sopratutto il reparto di attacco. Le società chiedono tantissimi soldi per un giocatore "solamente" perchè ha realizzato molti Gol o Assist nella stagione precedente. Non tutte le squadre, però, hanno una disponibilità economica elevata per comprare un attaccante già affermato. La nostra tesi è che esistano delle correlazioni, fra le statistiche sopracitate e le altre di gioco, che una squadra può andare a studiare per stimare il numero di Gol e Assist che può realizzare un attaccante. In questo modo una società non sarà costretta ad affidarsi al numero di Gol ed Assist e potrà comprare giocatori altrettanto validi ad un prezzo minore, risparmiando soldi.

DATA CLEANING

Prima di procedere con l'analisi bisogna fare un processo di 'data cleaning' ovvero una pulizia del dataframe, aggiungendo colonne che possono essere utili oppure togliendone altre che non vogliamo prendere in considerazione.

Eliminiamo cartellini gialli e rossi collezionati dai giocatori, in quanto rappresentano la disciplina di questi e non le loro capacità balistiche. Dopodiche eliminiamo le colonne relative ai calci piazzati in quanto rigori e punizioni hanno i propri specialisti scelti dall'allenatore, non sarebbe quindi equo considerare una statistica alla quale la maggior parte dei giocatori non prendono neanche parte.

Ovviamente scartiamo le celle relative al Nome del giocatore, la Squadra per cui gioca e la sua Nazionalità. Subito dopo togliamo quelle colonne che rappresentano "operazioni" fra altre statistiche, come ad esempio somme fra due dati, percentuali o rapporti, in quanto non sono statistiche di gioco (realizzate sul campo dal giocatore) ma create ad hoc per altri tipi di analisi non inerenti al nostro.

Queste statistiche non verranno mai considerate in questa parte del progetto.

Suddivideremo le statistiche del database in 4 categorie e le analizzeremo una ad una.

1) STATISTICHE DI IMPIEGO

La prima categoria riguarda le statistiche non di campo, quelle relative ad età, minutaggio e partite giocate

In [ ]:
#Crea 5 matrici colorate in cui ogni quadratino rappresenta l'indice di  correlazione fra le statistiche per annata
colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad','G+A', 'G-PK', 'PK',
       'PKatt', 'CrdY', 'CrdR', 'xG', 'PrgP', 'PrgR', 'Sh', 'SoT', 'SoT%',
       'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG', 'Cmp', 'Att', 'Cmp%', 'xA', 'KP',
       'PPA', 'CrsPA', 'SCA', 'GCA', 'Tkl', 'TklW', 'Mid 3rd', 'Att 3rd',
       'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%',
       'Carries', 'TotDist', '2CrdY', 'Fld', 'Off', 'Crs', 'Won', 'Lost',
       'Won%']

for file in files:
    
    data_frame = pd.read_excel(file)

    # Uso .drop per rimuovere dal data frame quelle statistiche che non voglio usare in questa analisi
    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr()  # Calcolo la matrice di correlazione

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year_mapping[file]}", zmin = -1, zmax = 1)
    # Ottengo in output il grafico di correlazione

    fig.show()

# Inserisco questa riga di codice in modo da poter visualizzare le colonne usate per l'analisi
print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')

Il codice stampa delle matrici interattive quadrate simmetriche dove ogni cella è colorata in base all'indice di correlazione delle statistiche scritte sulla riga e colonna corrispondente. Ovviamente la diagonale ha tutti i valori uguali ad uno in quanto indica la correlazione di una statistica con se stessa.
Diciamo che è interattiva in quanto mettendo il cursore su un quadrato vengono indicate le statistiche e l'indice fra di loro.
Sulla destra è riportata una barra che fa da legenda, con la quale si può associare al colore il valore dell'indice, vengono impostati estremi fissi e non variabili durante tutte l'analisi in modo che i colori possano aiutarci in una fase preliminare dell'analisi.

L'indice di correlazione è un numero compreso tra -1 e 1 che indica la relazione fra due variabili. Un indice di correlazione positivo indica che all'aumentare di una statistica anche l'altra lo fa. Mentre quello negativo indica che all'aumentare di una l'altra diminuisce. Più questo valore è vicino all'uno, in modulo, più la relazione è forte, più ci si avvicina allo zero più è debole

A noi importa analizzare gli indici delle statistiche con Gol ed Assist, quindi guardiamo le ultime due righe (colonne). 
Con la matrice possiamo farci un'idea preliminare, grazie ai colori, dell'indice di correlazione fra le statistiche.

In [ ]:
# Creo una funzione che restituisce l'elenco degli indci di correlazione di una statistica con le altre anno per anno

def correlazioni_annuali(stat):
    for file in files:
        data_frame = pd.read_excel(file)  

        data_frame.drop(columns=colonne_da_eliminare, inplace=True)

        corr_matrix = data_frame.corr()

        if stat in corr_matrix.columns:
            sorted_corr = corr_matrix[stat].sort_values(ascending=False)

            print(f"Anno {year_mapping[file]}: Le correlazioni per {stat} sono:\n{sorted_corr}\n")
        else:
            print(f"La statistica {stat} non è presente nel dataset dell'anno {year_mapping[file]}.")
        
# Creo una seconda funzione che mi dice l'indice di correlazione di due statistiche specifiche
# ed il cambiamento di questo nel corso delle stagioni

def correlazione(stat1,stat2):
 correlations = []

 for file in files:
    
    data_frame = pd.read_excel(file)

    data_frame.drop(columns=colonne_da_eliminare, inplace=True)

    corr_matrix = data_frame.corr()

    # Verifica se entrambe le statistiche esistono nella matrice di correlazione
    if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
        correlation_value = corr_matrix.loc[stat1, stat2]
        correlations.append((year_mapping[file], correlation_value))
        print(f"Anno {year_mapping[file]}: Indice {correlation_value}")
    else:
        correlations.append((year, None))

# Creazione del DataFrame per il grafico
 if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
    correlation_df = pd.DataFrame(correlations, columns=['Anno', 'Correlazione'])
    
    # Crea il grafico dell'andamento della correlazione nel tempo
    fig = px.line(correlation_df, x='Anno', y='Correlazione', title=f"Correlazione tra {stat1} e {stat2}", markers=True)
    fig.update_xaxes(dtick=1)
    fig.update_yaxes(range = [-1,1], dtick = 0.10)
    fig.show()
    print(f"Il massimo valore dell'indice fra {stat1} e {stat2} è {correlation_df['Correlazione'].max()} il valore minimo è {correlation_df['Correlazione'].min()}")
 else:
    print("Nessun grafico disponibile: non sono state calcolate correlazioni valide. Controlla l'input")

Ci aiutiamo con delle funzioni per plottare l'andamento di questi indici negli anni considerati.

In [ ]:
for column in data_frame.columns:
    if column not in ['Gls','Ast']:
     correlazione('Gls',column)

Notiamo che l'unica coppia che presenta massimo e minimo negativi è Gls e Min si presenta quindi una anticorrelazione. Per le altre 3 coppie si ha correlazione diretta in quanto massimo e minimo sono entrambi positivi.

Ragionando in valore assoluto, la coppia che risulta avere la correlazione più debole è Gls e Age nel 2020. Mentre quella più forte è Gls e Starts nel 2021.

In [ ]:
for column in data_frame.columns:
    if column not in ['Gls','Ast']:
     correlazione('Ast',column)

Anche in questo caso si presenta anticorrelazione con i minuti giocati, infatti anche qui il range è interamente a valori negativi. Tutte le altre hanno intervallo con massimo e minimo positivi e quindi correlazione diretta.

Passando al valore assoluto la coppia con la correlazione più debole è Ast e Min nel 2021. Quella più forte ad 1 è Ast e Starts nel 2020.

2) STATISTICHE DI OPPOSIZIONE

In questa parte ci occupiamo di analizzare le statistiche di opposizione, ovvero quelle che il riguardano il comportamento del giocatore quando è messo a confronto con un avversario. Analizzeremo dati come dribbling, contrasti e conduzioni palla. 

In questa parte son presenti statistiche sulla parte di campo in cui vengono registrati determinati dati. Le analizziamo separatamente in quanto vogliamo capire se c'è un impatto diverso legato all'area di campo in cui vengono registrati.

In [ ]:
correlations = []

colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad','Age', 'MP', 'Starts', 'Min', 'G+A', 'G-PK', 'PK',
       'PKatt', 'CrdY', 'CrdR', 'xG', 'PrgP', 'PrgR', 'Sh', 'SoT', 'SoT%',
       'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG', 'Cmp', 'Att', 'Cmp%', 'xA', 'KP',
       'PPA', 'CrsPA', 'SCA', 'GCA', 'Succ%','2CrdY', 'Off', 'Crs','Won%']

for file in files:
    
    data_frame = pd.read_excel(file)
    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr()

    year = year_mapping[file] 

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}", zmin = -1, zmax = 1)
    fig.show()
print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')

Visivamente possiamo notare come in questo blocco di statistiche gli indici siano più alti rispetto a quello precedente, essendo la griglia composta da colori più caldi. Quindi ci aspettiamo che i range di correlazione siano tutti positivi.

In [ ]:
for column in ['Mid 3rd', 'Att 3rd', 'Mid 3rd.1', 'Att 3rd.1']:
    correlazione('Gls',column)

Le prime due coppie sono relative ai contrasti, nella parte centrale di campo e quella offensiva. La seconda invece tratta i dribbling, anche qui divisi in parte centrale e offensiva. Per entrambe le categorie notiamo che gli intervalli degli indici sono più elevati per la parte offensiva. Quindi possiamo concludere che la parte di campo in cui vengono fatti dribbling e contrasti ha effettivamente un impatto sui Gol.

In [ ]:
for column in ['Mid 3rd', 'Att 3rd', 'Mid 3rd.1', 'Att 3rd.1']:
    correlazione('Ast',column)

Vediamo che i risultati sono simili a prima quindi il discorso è il medesimo che per i gol. Realizzare dribbling e contrasti nella parte offensiva di campo ha un impatto maggiore sugli assist piuttosto che farli a centrocampo.

In [ ]:
for column in data_frame.columns:
    if column not in ['Gls','Ast','Mid 3rd', 'Att 3rd', 'Mid 3rd.1', 'Att 3rd.1']:
     correlazione('Gls',column)

Vediamo che tutti i minimi sono positivi quindi siamo in presenza solamente di correlazioni dirette.
La coppia con corelazione più debole è Gls e TklW nel 2020 . Quella più forte è Gls e Touches nel 2023.

In [ ]:
for column in data_frame.columns:
    if column not in ['Gls','Ast','Mid 3rd', 'Att 3rd', 'Mid 3rd.1', 'Att 3rd.1']:
     correlazione('Ast',column)

Anche qui siamo solamente in presenza di correlazioni dirette. Rispetto ai gol si hanno correlazioni più forti, come in Carries che ha registrato la correlazione più forte con Ast nel 2021, ma anche alcune più deboli come quella di Won che ha registrato l'indice di correlazione con Ast più basso nel 2022.

3) STATISTICHE DI GIOCO

In questa parte andiamo ad analizzare le statistiche di creazione offensiva, quelle relative ai passaggi ai tiri e alla creazione di chance da gol o di tiro

In [ ]:
correlations = []
years = []

colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad','Age', 'MP', 'Starts', 'Min', 'G+A', 'G-PK', 'PK',
       'PKatt', 'CrdY', 'CrdR', 'SoT%', 'xG', 'xA',
       'G/Sh', 'G/SoT', 'FK', 'G-xG','Cmp%','Tkl', 'TklW', 'Mid 3rd', 'Att 3rd',
       'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%',
       'Carries', 'TotDist', '2CrdY', 'Fld', 'Won', 'Lost',
       'Won%']

for file in files:
    
    data_frame = pd.read_excel(file)
    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr()

    year = year_mapping[file] 
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}", zmin = -1, zmax = 1)
    fig.show()
    
print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')

In queste matrici vediamo colori molto caldi, quindi ci aspettiamo solo correlazioni dirette. Ad eccezione per una colonna con correlazione molto debole vicinissima allo zero sia per Gol che per Assist relativa a Dist, ovvero la distanza media di tiro. Risultato prevedibile. Analizziamola a parte per dimostrare ciò che abbiamo appena detto.

In [ ]:
for column in ['Gls','Ast']:
    correlazione('Dist',column)

Fra Dist e Gol l'intervallo degli indici di correlazione è negativo siamo in presenza di anticorrelazione molto debole.
Con gli Ast l'intervallo comprende valori negativi e positivi tutti vicini allo zero correlazione quindi, anche in questo caso, molto debole.

In [ ]:
for column in data_frame.columns:
    if column not in ['Gls','Ast','Dist']:
     correlazione('Gls',column)

Come anticipato, siccome tutti i minimi sono positivi, siamo in presenza di correlazioni dirette più o meno forti. La più forte è Gls e SoT nel 2021, il range è interamente molto vicino all'uno quindi siamo in presenza di una correlazione molto forte. Al contrario la correlazione è con CrsPA registrato nel 2019.

In [ ]:
for column in data_frame.columns:
    if column not in ['Gls','Ast','Dist']:
     correlazione('Ast',column)

Siamo in presenza di sole correlazioni dirette. Qua per gli assist la correlazione più forte è con GCA registrata nel 2021. Mentre la più debole è con Off registrato nel 2019.

4) GOL E ASSIST ATTESI

In questa parte finale di analisi ci concentriamo sul concetto specifico di Gol ed Assist attesi

I concetti di gol attesi (xG) e assist attesi (xA) sono metriche avanzate utilizzate nel calcio per misurare la qualità delle azioni da gol e degli assist, tenendo conto di fattori che influenzano la probabilità che si verifichi un gol o un assist. Il valore è compreso tra 0 e 1, dove un valore di 1 indica un evento che ha quasi certezza di verificarsi. mentre 0.1 rappresenta un tiro (ad esempio) molto difficile con poca probabilità di realizzazione.
Nello specifico, xG prende in considerazione aspetti come:
- Posizione del tiro: più vicino alla porta, più alta è la probabilità di segnare.
- Angolo di tiro: tiri da angoli difficili (ad esempio, tiri da fuori area o angoli stretti) hanno una probabilità inferiore di andare in gol.
- Posizione del portiere: se il portiere è fuori posizione o se c'è un errore difensivo, la probabilità di segnare aumenta.

Come per l'xG, anche l'xA si basa su vari fattori che influenzano la probabilità di trasformare un passaggio in un gol, come:
- Tipo di passaggio: passaggi facili da ricevere e controllare (come palloni rasoterra diretti) hanno una probabilità maggiore di diventare assist.
- Distanza dal gol: passaggi effettuati vicino alla porta o da posizioni centrali hanno un xA più alto.
- Posizione del ricevente: se il destinatario del passaggio è in una buona posizione per segnare, l'assist atteso sarà più alto.

In [ ]:
colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad','Age', 'MP', 'Starts', 'Min', 'G+A', 'G-PK', 'PK',
       'PKatt', 'CrdY', 'CrdR', 'SoT%',
       'G/Sh', 'G/SoT', 'FK', 'G-xG','Cmp%','Tkl', 'TklW', 'Mid 3rd', 'Att 3rd',
       'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%',
       'Carries', 'TotDist', '2CrdY', 'Fld','Won', 'Lost',
       'Won%']

In [ ]:
stat1 = 'Gls'
stat2 = 'xG'
correlazione(stat1,stat2)

Come potevamo immaginarci dalla spiegazione precedente il range è estremamente vicino ad 1, quindi la correlazione è molto forte fra queste due statistiche. Il risultato è molto comprensibile in quanto dietro a questi dati ci sono studi molto profondi e complessi realizzati anche con l'aiuto di intelligenze artificiali e database pienissimi di dati di miliardi di giocatori presi in diverse annate.

In [ ]:
stat1 = 'Ast'
stat2 = 'xA'
correlazione(stat1,stat2)

La correlazione anche in questo caso è forte. Lo è meno di quanto lo sono gli xG con i Gol in quanto per gli assist c'è la variabile relativa al compagno che può non riuscire a concretizzare un ottimo passaggio.

CONCLUSIONE

La ricerca conferma la nostra tesi. Esistono statistiche che presentano correlazioni forti con Gol ed Assist. Inoltre per ogni coppia di statistiche la differenza fra il minimo ed il massimo valore che assume l'indice mostra una costanza nella correlazione delle statistiche nel corso delle stagioni.

RETI NEURALI

Grazie al paragrafo precedente siamo stati in grado di determinare le correlazioni delle varie statistiche con gol e assist.

L'obiettivo di questa sezione è quello di usare i risultati trovati in precedenza per stimare il numero di Gol dei giocatori. 


Nelle prossime celle andremo ad impostare la rete neurale, importando i file ed addestrandola.

Le "incognite" del nostro lavoro, che vanno ad determinare il risultato, sono il numero di statistiche utilizzate nella rete neurale e quello di epoche di addestramento. Lavoreremo, quindi, con due cicli for per trovare la miglior combinazione di epoche e colonne che restituisca l'errore medio minore.

Ordiniamo le statistiche in base all'indice di correlazione con i Gol, così facendo andiamo ad inserire, per ogni iterazione del ciclo for, le statistiche in ordine di correlazione decrescente.

In [ ]:
f1 = "attaccanti_19.xlsx"
f2 = "attaccanti_20.xlsx"
f3 = "attaccanti_21.xlsx"
f4 = "attaccanti_22.xlsx"
f5 = "attaccanti_23.xlsx"

colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad','Ast','G+A', 'G-PK', 'PK','PKatt', 'CrdY', 'CrdR', 'SoT%','G/Sh',
'G/SoT', 'FK', 'G-xG', 'Cmp%', 'Succ%','2CrdY', 'Won%']

correlazioni_annuali('Gls')  # scopriamo l'ordine di correlazione per l'anno che ci interessa
colonne_ordinate = ["Gls", "xG", "SoT", "Sh", "GCA", "Starts", "PrgR", "Att 3rd.1", "SCA", "Touches", "Mid 3rd.1", "Carries", "KP", "Att",
                     "MP", "Cmp", "Fld", "xA", "PrgP", "Off", "TotDist", "Att.1", "PPA", "Att 3rd", "Succ", "Lost", "Won", 
                     "Tkl", "TklW", "Mid 3rd", "Crs", "CrsPA", "Age", "Dist", "Min"]
# Unisco i file Excel
df1 = pd.read_excel(f1)
df2 = pd.read_excel(f2).iloc[1:].reset_index(drop=True)
df3 = pd.read_excel(f3).iloc[1:].reset_index(drop=True)
df4 = pd.read_excel(f4).iloc[1:].reset_index(drop=True)
df_unito = pd.concat([df1, df2, df3, df4], ignore_index=True)
df_unito = df_unito[colonne_ordinate]

df5 = pd.read_excel(f5)
df5 = df5.drop(columns=colonne_da_eliminare, errors='ignore')
df5 = df5[colonne_ordinate]

Si fa notare che verranno prese in considerazione solamente le prime 25 statistiche (in ordine di correlazione) poichè notiamo che è il numero di colonne per cui l'errore inizia ad aumentare e non oscillare come avviene per gli intervalli precedenti.

In [ ]:
# Pre-elaborazione dei dati
scaler = StandardScaler()

# Variabili per risultati complessivi
errori_predizione = []
num_colonne_usate = []
minimi_errori_epoch = []  # Lista per tenere traccia dei minimi RMSE per epoche

In [ ]:
#for num_col in range(2, 25):  
    print(f"Training usando le prime {num_col} colonne.")
    
    X = df_unito.iloc[:, 1:num_col]  # Input
    y = df_unito.iloc[:, :1]         # Output
    
    X_scaled = scaler.fit_transform(X)

    X_train, y_train = X_scaled, y
    
    # Variabili per memorizzare il minimo errore e il numero di epoche
    best_rmse_prediction = float('inf')
    best_epochs = 0
    
    for epochs in range(100, 1201, 100):
    
        model = tf.keras.Sequential([tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
            tf.keras.layers.Dense(64, activation='relu'), tf.keras.layers.Dense(y_train.shape[1])])

        model.compile(optimizer='adam', loss='mean_squared_error')

        history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_split=0.2, verbose=0)
    
        X_new = df5[X.columns]
        X_new_scaled = scaler.transform(X_new)
        
        y_pred = model.predict(X_new_scaled, verbose=0)
        
        y_pred = np.clip(y_pred, 0, None)
        
        y_actual = df5['Gls'].values  
        mse_prediction = mean_squared_error(y_actual, y_pred)
        rmse_prediction = sqrt(mse_prediction) 
        
        # Cerco il minimo RMSE di predizione
        if rmse_prediction < best_rmse_prediction:
            best_rmse_prediction = rmse_prediction
            best_epochs = epochs 
        
    # Aggiungi i migliori risultati trovati per questa configurazione
    errori_predizione.append(best_rmse_prediction)
    num_colonne_usate.append(num_col)
    minimi_errori_epoch.append((best_rmse_prediction, best_epochs))
    
    # Stampa il miglior risultato per colonne
    print(f"Con {num_col} colonne:")
    print(f"Minimo RMSE Predizione: {best_rmse_prediction} (Epochs: {best_epochs})")

    # Stampa i migliori risultati per ogni configurazione
    for i, (rmse, epochs) in enumerate(minimi_errori_epoch):
     print(f"Per {num_colonne_usate[i]} colonne, il miglior RMSE di predizione è {rmse} con {epochs} epoche.")

Il for è stato eseguito su un altro computer per via dell'elevata durata del ciclo, basti pensare il numero di epoche totali corrispondente a 195000. Vengono riportati i risultati ottenuti.

Per 2 colonne, il miglior RMSE di predizione è 1.907446850944147 con 100 epoche.
Per 3 colonne, il miglior RMSE di predizione è 1.8851412553280629 con 700 epoche.
Per 4 colonne, il miglior RMSE di predizione è 1.941661028482543 con 200 epoche.
Per 5 colonne, il miglior RMSE di predizione è 2.084774711957049 con 100 epoche.
Per 6 colonne, il miglior RMSE di predizione è 2.0822575432714294 con 100 epoche.
Per 7 colonne, il miglior RMSE di predizione è 1.9618783343361386 con 100 epoche.
Per 8 colonne, il miglior RMSE di predizione è 2.022257559384076 con 100 epoche.
Per 9 colonne, il miglior RMSE di predizione è 2.048596509083856 con 100 epoche.
Per 10 colonne, il miglior RMSE di predizione è 2.001122898359071 con 100 epoche.
Per 11 colonne, il miglior RMSE di predizione è 1.9927146743435766 con 100 epoche.
Per 12 colonne, il miglior RMSE di predizione è 1.9331462995970639 con 100 epoche.
Per 13 colonne, il miglior RMSE di predizione è 1.9759040473452913 con 100 epoche.
Per 14 colonne, il miglior RMSE di predizione è 1.9982565406445278 con 100 epoche.
Per 15 colonne, il miglior RMSE di predizione è 1.9369245286742989 con 100 epoche.
Per 16 colonne, il miglior RMSE di predizione è 1.939918307149599 con 100 epoche.
Per 17 colonne, il miglior RMSE di predizione è 2.0128151752158936 con 100 epoche.
Per 18 colonne, il miglior RMSE di predizione è 2.0199483868915977 con 100 epoche.
Per 19 colonne, il miglior RMSE di predizione è 2.087056068104932 con 100 epoche.
Per 20 colonne, il miglior RMSE di predizione è 2.0493591286211474 con 100 epoche.
Per 21 colonne, il miglior RMSE di predizione è 2.0654617693189468 con 100 epoche.
Per 22 colonne, il miglior RMSE di predizione è 1.9711563695446321 con 300 epoche.
Per 23 colonne, il miglior RMSE di predizione è 2.083283808534682 con 100 epoche.
Per 24 colonne, il miglior RMSE di predizione è 2.0020954979667303 con 100 epoche.

Creiamo il grafico inserendo i dati manualmente per non far girare il codice che impiegherebbe diverse ore.

In [ ]:
num_colonne = list(range(2, 25))
rmse_values = [
    1.907446850944147, 1.8851412553280629, 1.941661028482543, 2.084774711957049, 2.0822575432714294, 1.9618783343361386, 2.022257559384076, 2.048596509083856, 
    2.001122898359071, 1.9927146743435766, 1.9331462995970639, 1.9759040473452913, 1.9982565406445278, 1.9369245286742989, 1.939918307149599, 2.0128151752158936, 
    2.0199483868915977, 2.087056068104932, 2.0493591286211474, 2.0654617693189468, 1.9711563695446321, 2.083283808534682, 2.0020954979667303]

epochs_values = [100, 700, 200, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 300, 100, 100]

fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.set_xlabel('Numero di colonne')
ax1.set_ylabel('Errore (RMSE)')
ax1.plot(num_colonne, rmse_values, marker='o', color='b', label='Errore (RMSE)')
ax1.tick_params(axis='y')
ax1.legend(loc='upper left')

ax2 = ax1.twinx()
ax2.set_ylabel('Epoche ottimali')
ax2.plot(num_colonne, epochs_values, marker='s', color='red', label='Epoche ottimali')
ax2.tick_params(axis='y')
ax2.legend(loc='upper right')

plt.title('Errore e epoche ottimali al variare del numero di colonne')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

plt.show()

La migliore coppia di colonne ed epoche è (3;700). Per trovare un risultato più preciso usiamo le prime 3 colonne e lavoriamo con un range più piccolo di epoche ad intervalli minori.

In [ ]:
# prime 3 colonne come input
num_col = 3
print(f"Training usando le prime {num_col} colonne.")

X = df_unito.iloc[:, 1:num_col]  
y = df_unito.iloc[:, :1]        

X_scaled = scaler.fit_transform(X)

X_train, y_train = X_scaled, y

best_rmse_prediction = float('inf')
best_epochs = 0

for epochs in range(640, 801, 20):
    model = tf.keras.Sequential([tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),tf.keras.layers.Dense(y_train.shape[1])])

    model.compile(optimizer='adam', loss='mean_squared_error')
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_split=0.2, verbose=0)
    
    X_new = df5[X.columns]
    X_new_scaled = scaler.transform(X_new)
    
    y_pred = model.predict(X_new_scaled, verbose=0)
    
    y_pred = np.clip(y_pred, 0, None)
    
    y_actual = df5['Gls'].values
    mse_prediction = mean_squared_error(y_actual, y_pred)
    rmse_prediction = sqrt(mse_prediction)  
    
    if rmse_prediction < best_rmse_prediction:
        best_rmse_prediction = rmse_prediction
        best_epochs = epochs 

print(f"Con {num_col} colonne:")
print(f"Minimo RMSE Predizione: {best_rmse_prediction} (Epochs: {best_epochs})")

Trovato il miglior numero di epoche per le colonne desiderate, plottiamo un grafico nel quale rappresentiamo visiamente le predizioni della nostra rete neurale, in confronto alla bisettrice che rappresenta il valore esatto dei Gol.

In [ ]:
y_actual=np.array([y_actual])
y_pred=np.array([y_pred])
y_pred = y_pred.squeeze()
y_actual = y_actual.squeeze()

fuori_dalla_linea = (y_actual < y_pred - best_rmse_prediction) | (y_actual > y_pred + best_rmse_prediction)

y_actual_fuori = y_actual[fuori_dalla_linea]
y_pred_fuori = y_pred[fuori_dalla_linea]

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_actual, y_pred, alpha=0.7, edgecolors='k', label='Previsioni')
plt.plot([0, max(y_actual)], [0, max(y_actual)], color = 'k', label='Linea Perfetta')
plt.xlabel("Goal Reali")
plt.ylabel("Goal Previsti")
plt.title("Confronto tra Goal Reali e Goal Previsti")
plt.legend()
plt.grid(True)
plt.show()

Notiamo con soddisfazione che la nostra rete neurale restituisce valori coerenti con quelli con il grafico desiderato y = x. Andiamo ora a valutare l'efficienza, ovvero il numero di giocatori la cui barra di errore interseca la retta.

In [ ]:
plt.figure(figsize=(10, 6))

plt.errorbar(y_actual, y_pred, fmt='o', ecolor='green', color='green', alpha = 0.5, label='Previsioni accettabili')

plt.errorbar(y_actual_fuori, y_pred_fuori, fmt='o', color='red', alpha = 0.5, label='Previsioni non accettabili')

plt.plot([0, max(y_actual)], [0, max(y_actual)], color='black', label='Linea Perfetta')

plt.plot([0, max(y_actual)], [rmse_prediction, max(y_actual) + rmse_prediction], color='k', linestyle='--', label='Linea +RMSE')
plt.plot([0, max(y_actual)], [-rmse_prediction, max(y_actual) - rmse_prediction], color='k', linestyle='--', label='Linea -RMSE')

plt.xlabel("Goal Reali")
plt.ylabel("Goal Previsti")
plt.title("Confronto tra Goal Reali e Goal Previsti con Linee RMSE")
plt.legend()
plt.grid(True)
plt.show()

num_fuori_dalla_linea = np.sum(fuori_dalla_linea)

print(f"Numero di giocatori fuori dal range: {num_fuori_dalla_linea}")
print(f"Numero di giocatori nel range: {194 - num_fuori_dalla_linea}")

La maggior parte dei punti verdi si concentra nella parte bassa del grafico, con valori reali e previsti intorno a 0-10.
Gli outlier (punti rossi) appaiono più frequenti in corrispondenza di valori elevati dei goal reali o previsti (ad esempio, sopra 15-20), suggerendo che il modello può avere difficoltà con predizioni per valori estremi.
Il modello sembra funzionare meglio per i valori bassi di goal reali e previsti, ma perde precisione man mano che i valori aumentano.
Una possibile spiegazione è che il dataset contenga meno osservazioni con goal elevati o che il modello non riesca a generalizzare per queste situazioni.